In [1]:
!wget -q -nc https://raw.githubusercontent.com/pseprivamirakbarnejad/cmput206lab/master/Lab8/parabola_horz.png
!ls

parabola_horz.png  sample_data


In [0]:
import numpy as np
import cv2
from sklearn.cluster import KMeans
from skimage.color import gray2rgb
import scipy.ndimage.filters as filters
from matplotlib import pyplot as plt

# Compute accumulator for parabola detection

A parabola with vertical axis is defined by the equation:

$(x − h)^2 = 4a(y − k)$

while one with horizontal axis is defined as:

$(y − k)^2 = 4a(x − h)$

where *`(h, k)`* is the center and *`a`* controls the width or spread of the parabola.

To simplify the problem, both parabolas have horizontal axes and are centered vertically so you only need to find *`a`* and *`h`* using the second equation.

1. choose suitable minimum and maximum values for *`a`* and *`h`* along with the number of values *`num_a`* and *`num_h`* to sample the respective intervals. Note that the two parabolas have different orientations so that *`a < 0`* for one and *`a > 0`* for the other.
2. create vectors *`a_vec`* and *`h_vec`* of sizes *`num_a`* and *`num_h`* respectively containing all possible values of *`a`* and *`h`* (hint: *`np.linspace`*)
3. create the accumulator array of size *`num_a`* $\times$ *`num_h`* containing all zeros and of type *uint64*
4. find locations of all edge pixels in the input image (hint: *`np.nonzero`*)
5. iterate over the edge pixels and fill up the accumulator array

  5.1. for each edge pixel *`x, y`*, iterate over all values of *`a`* and calculate the *`h`* corresponding to each *`a`* using the equation
$(y − k)^2 = 4a(x − h)$

  5.2. find the index of this `h` within `h_vec` which will give the column index for accumulator while the index of *`a`* within *`a_vec`* gives the corresponding row index.

  5.3. increment the accumulator entry corresponding to this row and column index by 1
6. filled accumulator array should look similar to this:
<img src="https://raw.githubusercontent.com/pseprivamirakbarnejad/cmput206lab/master/Lab8/parabola_horz_accumulator.png">



In [0]:
def hough_parabola_horz(img, k):
  """
  add your code here
  """

  a_min=-1500
  a_max=1500
  h_min=-1500
  h_max=1500
  
  num_a=40
  num_h=40
  

  a_vec = np.linspace(a_min,a_max, num_a)
  h_vec = np.linspace(h_min, h_max, num_h)
  
  accumulator = np.zeros((num_a, num_h), dtype=np.uint64)
  height, width = img.shape

  edgePixels = np.nonzero(img)
  
  for i in range(edgePixels):
    for j in range(a_vec):
      x= edgePixels[0][i]
      y=edgePixels[1][i]
      a = a_vec[y]

      h = -(pow(y-k,2)/(4*a)-x)
      

      hIdx = int((h-h_min)/(h_max-h_min))* num_h

      if (hIdx >=0 and hIdx <num_h):
        accumulator[j][hIdx] = accumulator[j][hIdx] +1



  return accumulator, a_vec, h_vec

# Find the locations of two distinct local maxima in the accumulator array

1. choose a suitable integer *`n`* such that top *`n`* maxima will include pixels from both maxima

2. find the top *`n`* maxima (hint: *`np.argsort`*) and fill in *`max_idx`* which is a *`2 x n`* array containing the *`x, y`* coordinates of the *n* maxima. A binary image showing the maxima should look like this:
<img src="https://raw.githubusercontent.com/pseprivamirakbarnejad/cmput206lab/master/Lab8/parabola_horz_maxima.png">

3. use KMeans clustering to find 2 clusters among these *`n`* points:

    https://towardsdatascience.com/machine-learning-algorithms-part-9-k-means-example-in-python-f2ad05ed5203

4. obtain the centers of the two clusters (hint: *`kmeans.cluster_centers_`*) that correspond to the indices of the two *`a, h`* pairs

5. extract the corresponding *`a, h`* values from *`a_vec`* and *`h_vec`*

6. the detected parabolas when drawn on the input image should closely overlay the source parabolas:

<img src="https://raw.githubusercontent.com/pseprivamirakbarnejad/cmput206lab/master/Lab8/parabola_horz_detections.png">


In [0]:
def find_maxima(accumulator, a_vec, h_vec):
  """
  add your code here
  """
  n=400
  argsortVal = np.argsort(accumulator)

  max_idx = np.zeros(2,n)

  kVal = KMeans(n_clusters=2,  max_iter=300, n_init=10, random_state=0)

  a,h = kmeans.cluster_centers_
  

  maxima_img = np.zeros_like(accumulator).astype(np.uint8)
  for (x, y) in max_idx:
      maxima_img[x, y] = 255

  plt.figure()
  plt.title('Accumulator Maxima')
  plt.imshow(maxima_img, cmap='gray')
  plt.axis('off')

  return a1, h1, a2, h2

**Do not change anything in the following code**

In [0]:
def draw_parabola_horz(t_vec, num_t, a, h, k, img_w, img_h, out_img, col):
    for t_idx in range(num_t):
        t = t_vec[t_idx]

        y = 2 * a * t + k
        x = a * t * t + h

        if img_w > x > 0 and img_h > y > 0:
            out_img[int(y), int(x)] = col

        y = k - 2 * a * t

        if img_w > x > 0 and img_h > y > 0:
            out_img[int(y), int(x)] = col

In [6]:
img = cv2.imread('parabola_horz.png', 0)
img_h, img_w = img.shape

# vertically centered parabolas
k = img_h / 2

accumulator, a_vec, h_vec = hough_parabola_horz(img, k)

a1, h1, a2, h2 = find_maxima(accumulator, a_vec, h_vec)

t_min, t_max, num_t = 0, 100, 10000
t_vec = np.linspace(t_min, t_max, num_t)

detections_img = gray2rgb(img)
draw_parabola_horz(t_vec, num_t, a1, h1, k, img_w, img_h, detections_img, (0, 255, 0))
draw_parabola_horz(t_vec, num_t, a2, h2, k, img_w, img_h, detections_img, (0, 255, 0))

accumulator_img = accumulator.astype(np.float32) / np.amax(accumulator)

plt.figure()
plt.title('Input image')
plt.imshow(img, cmap='gray')
plt.axis('off')

plt.figure()
plt.title('Accumulator')
plt.imshow(accumulator_img, cmap='gray')
plt.axis('off')

plt.figure()
plt.title('Detected Parabolas')
plt.imshow(detections_img, cmap='gray')
plt.axis('off')

TypeError: ignored